In [1]:
import pandas as pd
import mysql.connector as sql
import streamlit as st
import plotly.express as px
import os
import json
from streamlit_option_menu import option_menu
from PIL import Image

In [6]:
Mydb = sql.connect(host = "localhost",
                   user = "root",
                   password = "your password",
                   database = "phonepe")
mycursor = Mydb.cursor(buffered=True)
cursor = Mydb.cursor()

In [3]:
Path_1 = "C:/Users/dines/Desktop/capston_Guvi/pulse/data/aggregated/transaction/country/india/state/"
agg_trans_list = os.listdir(Path_1)

columns_1 = {'State': [], 'Year': [], 'Quarter': [], 'Transaction_type': [], 'Transaction_count': [],
            'Transaction_amount': []}
for state in agg_trans_list:
    cur_state = Path_1 + state + "/"
    agg_year_list = os.listdir(cur_state)
    
    for year in agg_year_list:
        cur_year = cur_state + year + "/"
        agg_file_list = os.listdir(cur_year)
        
        for file in agg_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            A = json.load(data)
            
            for i in A['data']['transactionData']:
                name = i['name']
                count = i['paymentInstruments'][0]['count']
                amount = i['paymentInstruments'][0]['amount']
                columns_1['Transaction_type'].append(name)
                columns_1['Transaction_count'].append(count)
                columns_1['Transaction_amount'].append(amount)
                columns_1['State'].append(state)
                columns_1['Year'].append(year)
                columns_1['Quarter'].append(int(file.strip('.json')))
                
df_agg_trans = pd.DataFrame(columns_1)
df_agg_trans

,State,Year,Quarter,Transaction_type,Transaction_count,Transaction_amount
0,andaman-&-nicobar-islands,2018,1,Recharge & bill payments,4200,1.845307e+06
1,andaman-&-nicobar-islands,2018,1,Peer-to-peer payments,1871,1.213866e+07
2,andaman-&-nicobar-islands,2018,1,Merchant payments,298,4.525072e+05
3,andaman-&-nicobar-islands,2018,1,Financial Services,33,1.060142e+04
4,andaman-&-nicobar-islands,2018,1,Others,256,1.846899e+05
...,...,...,...,...,...,...
4129,west-bengal,2023,3,Merchant payments,296965002,2.004404e+11
4130,west-bengal,2023,3,Peer-to-peer payments,271150280,8.310501e+11
4131,west-bengal,2023,3,Recharge & bill payments,63055684,3.920985e+10
4132,west-bengal,2023,3,Financial Services,389751,4.249856e+08


In [43]:
# df_agg_trans.to_csv('Aggregated_Trans.csv',index=False)

In [8]:
mycursor.execute("create table aggregated_Trans(State varchar(100), Year int, Quarter int, Transaction_type varchar(100), Transaction_count int, Transaction_amount double)")
for i,row in df_agg_trans.iterrows():
    sql = "INSERT INTO aggregated_Trans VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    Mydb.commit()

In [4]:
Path_2 = "C:/Users/dines/Desktop/capston_Guvi/pulse/data/aggregated/user/country/india/state/"
agg_user_list = os.listdir(Path_2)

columns_2 = {'State': [], 'Year': [], 'Quarter': [], 'Brands': [], 'Count': [], 'Percentage': []}
for state in agg_user_list:
    cur_state = Path_2 + state + "/"
    agg_year_list = os.listdir(cur_state)
    
    for year in agg_year_list:
        cur_year = cur_state + year + "/"
        agg_file_list = os.listdir(cur_year)

        for file in agg_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            B = json.load(data)
            try:
                for i in B["data"]["usersByDevice"]:
                    brand_name = i["brand"]
                    counts = i["count"]
                    percents = i["percentage"]
                    columns_2["Brands"].append(brand_name)
                    columns_2["Count"].append(counts)
                    columns_2["Percentage"].append(percents)
                    columns_2["State"].append(state)
                    columns_2["Year"].append(year)
                    columns_2["Quarter"].append(int(file.strip('.json')))
            except:
                pass
df_agg_user = pd.DataFrame(columns_2)
df_agg_user

,State,Year,Quarter,Brands,Count,Percentage
0,andaman-&-nicobar-islands,2018,1,Xiaomi,1665,0.247033
1,andaman-&-nicobar-islands,2018,1,Samsung,1445,0.214392
2,andaman-&-nicobar-islands,2018,1,Vivo,982,0.145697
3,andaman-&-nicobar-islands,2018,1,Oppo,501,0.074332
4,andaman-&-nicobar-islands,2018,1,OnePlus,332,0.049258
...,...,...,...,...,...,...
6727,west-bengal,2022,1,Lenovo,330017,0.015056
6728,west-bengal,2022,1,Infinix,284678,0.012987
6729,west-bengal,2022,1,Asus,280347,0.012790
6730,west-bengal,2022,1,Apple,277752,0.012671


In [11]:
# df_agg_user.to_csv("Aggregated_User.csv",index=False)

In [12]:
mycursor.execute("create table Aggregate_User (State varchar(100), Year int, Quarter int, Brands varchar(100), Count int, Percentage double)")
for i,row in df_agg_user.iterrows():
    sql = "INSERT INTO Aggregate_User VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql,tuple(row))
    Mydb.commit()

In [5]:
path_3 = "C:/Users/dines/Desktop/capston_Guvi/pulse/data/map/transaction/hover/country/india/state/"
map_Tran_List = os.listdir(path_3)
columns_3 = {'State':[], 'Year':[], 'Quarter':[], 'District':[], 'Count':[], 'Amount':[]}
for state in map_Tran_List:
    cur_state = path_3 + state + "/"
    map_year_list =os.listdir(cur_state)
    
    for year in map_year_list:
        cur_year =  cur_state+ year + "/"
        map_file_list = os.listdir(cur_year)
        
        for file in map_file_list:
            cur_file = cur_year +file
            data = open(cur_file,'r')
            c = json.load(data)
            
            for i in c["data"]["hoverDataList"]:
                district = i["name"]
                count = i["metric"][0]["count"]
                amount = i["metric"][0]["amount"]
                columns_3["District"].append(district)
                columns_3["Count"].append(count)
                columns_3["Amount"].append(amount)
                columns_3['State'].append(state)
                columns_3['Year'].append(year)
                columns_3['Quarter'].append(int(file.strip('.json')))
Map_Trans = pd.DataFrame(columns_3)                
Map_Trans

,State,Year,Quarter,District,Count,Amount
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,442,9.316631e+05
1,andaman-&-nicobar-islands,2018,1,south andaman district,5688,1.256025e+07
2,andaman-&-nicobar-islands,2018,1,nicobars district,528,1.139849e+06
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,825,1.317863e+06
4,andaman-&-nicobar-islands,2018,2,south andaman district,9395,2.394824e+07
...,...,...,...,...,...,...
16827,west-bengal,2023,3,nadia district,30146530,5.787869e+10
16828,west-bengal,2023,3,birbhum district,17597047,3.090737e+10
16829,west-bengal,2023,3,purba medinipur district,34244893,6.813718e+10
16830,west-bengal,2023,3,maldah district,28539407,5.299766e+10


In [15]:
# Map_Trans.to_csv("Map_Trans.csv",index=False)

In [19]:
mycursor.execute("create table Map_Trans (State varchar(100),Year int, Quarter int, District varchar(100), Count int, Amount double)")
for i,row in Map_Trans.iterrows():
    sql = "INSERT INTO Map_Trans VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql,tuple(row))
    Mydb.commit()

In [6]:
Path_4 = "C:/Users/dines/Desktop/capston_Guvi/pulse/data/map/user/hover/country/india/state/"
map_user = os.listdir(Path_4)
Columns_4  = {'State':[], 'Year':[], 'Quarter':[], 'District':[], 'RegisteredUser':[], 'AppOpens':[]}
for state in map_user:
    cur_state = Path_4 + state + "/"
    map_year_list = os.listdir(cur_state)
    for year in map_year_list:
        cur_year = cur_state+year+"/"
        map_list = os.listdir(cur_year)
        for file in map_list:
            cur_file = cur_year+file
            data = open(cur_file,'r')
            d = json.load(data)
            for i in d["data"]["hoverData"].items():
                district = i[0]
                registereduser = i[1]["registeredUsers"]
                appOpens = i[1]['appOpens']
                Columns_4["District"].append(district)
                Columns_4["RegisteredUser"].append(registereduser)
                Columns_4["AppOpens"].append(appOpens)
                Columns_4['State'].append(state)
                Columns_4['Year'].append(year)
                Columns_4['Quarter'].append(int(file.strip('.json')))
data_map_User = pd.DataFrame(Columns_4)
data_map_User

,State,Year,Quarter,District,RegisteredUser,AppOpens
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,632,0
1,andaman-&-nicobar-islands,2018,1,south andaman district,5846,0
2,andaman-&-nicobar-islands,2018,1,nicobars district,262,0
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,911,0
4,andaman-&-nicobar-islands,2018,2,south andaman district,8143,0
...,...,...,...,...,...,...
16831,west-bengal,2023,3,nadia district,1544294,12674328
16832,west-bengal,2023,3,birbhum district,974363,18961683
16833,west-bengal,2023,3,purba medinipur district,1530289,18593769
16834,west-bengal,2023,3,maldah district,1091634,28020285


In [22]:
# data_map_User.to_csv("Map_User_data.csv",index=False)

In [23]:
mycursor.execute("create table Map_User (State varchar(100), Year int, Quarter int, District varchar(100), RegisteredUser int, AppOpens int)")
for i,row in data_map_User.iterrows():
    sql = ("INSERT INTO Map_User VALUES (%s,%s,%s,%s,%s,%s)")
    mycursor.execute(sql,tuple(row))
    Mydb.commit()

In [7]:
Path_5 = "C:/Users/dines/Desktop/capston_Guvi/pulse/data/top/transaction/country/india/state/"
top_trans_list = os.listdir(Path_5)
Columns_5 = {'State': [], 'Year': [], 'Quarter': [], 'Pincode': [], 'Transaction_count': [],'Transaction_amount': []}
for state in top_trans_list:
    cur_state = Path_5 + state + "/"
    top_year_list = os.listdir(cur_state)
    for year in top_year_list:
        cur_year = cur_state + year + "/"
        top_file_list = os.listdir(cur_year)
        for file in top_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            A = json.load(data)
            for i in A['data']['pincodes']:
                name = i['entityName']
                count = i['metric']['count']
                amount = i['metric']['amount']
                Columns_5['Pincode'].append(name)
                Columns_5['Transaction_count'].append(count)
                Columns_5['Transaction_amount'].append(amount)
                Columns_5['State'].append(state)
                Columns_5['Year'].append(year)
                Columns_5['Quarter'].append(int(file.strip('.json')))
df_top_trans = pd.DataFrame(Columns_5)
df_top_trans

,State,Year,Quarter,Pincode,Transaction_count,Transaction_amount
0,andaman-&-nicobar-islands,2018,1,744101,1622,2.769298e+06
1,andaman-&-nicobar-islands,2018,1,744103,1223,2.238042e+06
2,andaman-&-nicobar-islands,2018,1,744102,969,3.519060e+06
3,andaman-&-nicobar-islands,2018,1,744105,685,1.298561e+06
4,andaman-&-nicobar-islands,2018,1,744104,340,1.039715e+06
...,...,...,...,...,...,...
8205,west-bengal,2023,3,700001,3536928,7.193522e+09
8206,west-bengal,2023,3,721301,3464729,4.871566e+09
8207,west-bengal,2023,3,700039,3405551,5.095825e+09
8208,west-bengal,2023,3,732125,3347165,5.353831e+09


In [26]:
# df_top_trans.to_csv("Top_Trans_data.csv",index=False)

In [27]:
mycursor.execute("create table Top_trans (State varchar(100), Year int, Quarter int, Pincode int, Transaction_count int, Transaction_Amount double)")
for i,row in df_top_trans.iterrows():
    sql = ("INSERT INTO Top_trans VALUES (%s,%s,%s,%s,%s,%s)")
    mycursor.execute(sql,tuple(row))
    Mydb.commit()

In [8]:
Path_6 = "C:/Users/dines/Desktop/capston_Guvi/pulse/data/top/user/country/india/state/"
user_data = os.listdir(Path_6)
Columns_6 = {'State':[], 'Year':[], 'Quarter':[], 'Pincode':[], 'RegisteredUsers':[]}
for state in user_data:
    cur_state=Path_6 + state + "/"
    Year_data = os.listdir(cur_state)
    for year in Year_data:
        cur_year =cur_state + year + "/"
        Top_file = os.listdir(cur_year)
        for file in Top_file:
            cur_file = cur_year+file
            data = open(cur_file,'r')
            Data = json.load(data)
            for i in Data['data']['pincodes']:
                name = i['name']
                registeredUsers = i['registeredUsers']
                Columns_6['Pincode'].append(name)
                Columns_6['RegisteredUsers'].append(registeredUsers)
                Columns_6['State'].append(state)
                Columns_6['Year'].append(year)
                Columns_6['Quarter'].append(int(file.strip('.json')))
Top_user_Data = pd.DataFrame(Columns_6)
Top_user_Data

,State,Year,Quarter,Pincode,RegisteredUsers
0,andaman-&-nicobar-islands,2018,1,744103,1608
1,andaman-&-nicobar-islands,2018,1,744101,1108
2,andaman-&-nicobar-islands,2018,1,744105,1075
3,andaman-&-nicobar-islands,2018,1,744102,1006
4,andaman-&-nicobar-islands,2018,1,744104,272
...,...,...,...,...,...
8206,west-bengal,2023,3,700015,122316
8207,west-bengal,2023,3,742304,119245
8208,west-bengal,2023,3,721101,118798
8209,west-bengal,2023,3,700150,114987


In [30]:
# Top_user_Data.to_csv("Top_User_data.csv",index=False)

In [31]:
mycursor.execute("create table Top_User (State varchar(100), Year int, Quarter int, Pincode int, RegisteredUsers int)")
for i,row in Top_user_Data.iterrows():
    sql = ("INSERT INTO Top_User VALUES (%s,%s,%s,%s,%s)")
    mycursor.execute(sql,tuple(row))
    Mydb.commit()

In [32]:
mycursor.execute("show tables")
mycursor.fetchall()

[('aggregate_user',),
 ('aggregated_trans',),
 ('map_trans',),
 ('map_user',),
 ('top_trans',),
 ('top_user',)]

In [9]:
df_agg_trans['State'] = df_agg_trans['State'].replace({'himachal-pradesh':'Himachal Pradesh','dadra-&-nagar-haveli-&-daman-&-diu':'Dadra and Nagar Haveli and Daman and Diu',\
                                     'gujarat':'Gujarat','madhya-pradesh':'Madhya Pradesh','arunachal-pradesh':'Arunachal Pradesh',\
                                     'meghalaya':'Meghalaya','jharkhand':'Jharkhand','assam':'Assam','andhra-pradesh':'Andhra Pradesh',\
                                     'manipur':'Manipur','mizoram':'Mizoram','ladakh':'Ladakh','chhattisgarh':'Chhattisgarh','tripura':'Tripura',\
                                     'tamil-nadu':'Tamil Nadu','uttarakhand':'Uttarakhand','bihar':'Bihar','goa':'Goa','Kerala':'Kerala',\
                                     'rajasthan':'Rajasthan','haryana':'Haryana','nagaland':'Nagaland','odisha':'Odisha','uttar-pradesh':'Uttar Pradesh',\
                                     'west-bengal':'West Bengal','andaman-&-nicobar-islands':'Andaman & Nicobar','telangana':'Telangana','punjab':'Punjab',\
                                     'delhi':'Delhi','karnataka':'Karnataka','jammu-&-kashmir':'Jammu & Kashmir','maharashtra':'Maharashtra','chandigarh':'Chandigarh',\
                                     'puducherry':'Puducherry','sikkim':'Sikkim','lakshadweep':'Lakshadweep'})

In [ ]:
state_names=df_agg_trans['State'].unique()
state_names_df = pd.DataFrame({'state': state_names})
state_names_df.to_csv('state_names.csv', index=False)